# Monthly Aggregation of Citi Bike Trip Data
---
Tableau Public has a max allowance of 15,000,000 rows of data.  An entire year's worth of Citi Bike data was preferred, but that totalled over 18,000,000 rows.  The purpose of this notebook was to aggregate each month's data per bike station and birth year (saved to separate CSVs) so that visualizations could be created for an entire year's worth of data in Tableau.

This notebook reads in a CSV of Citi Bike's trip history data (found at https://www.citibikenyc.com/system-data), then aggregates the monthly totals of each start and end bike station, and each birth year.  The notebook had to be run for each CSV.  I'm sure there's a solution where the CSVs can be read and written in sequence, but I was short on time, so I opted for the quick, dirty, manual way of changing the date with variables at the top of the notebook.

### Dependencies

In [1]:
import pandas as pd
import os

### Prefix and Date Variables

In [2]:
# A prefix of "JC-" signifies data from New Jersey.  The prefix is blank for New York data.
prefix = "JC-"
year = "2019"
month = "03"

### Read in the CSV

In [3]:
df = pd.read_csv(os.path.join("data",f"{prefix}{year}{month}-citibike-tripdata.csv"))
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,306,2019-03-01 06:47:08.2110,2019-03-01 06:52:14.7150,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,26272,Subscriber,1989,1
1,123,2019-03-01 07:30:53.0220,2019-03-01 07:32:56.7770,3183,Exchange Place,40.716247,-74.033459,3184,Paulus Hook,40.714145,-74.033552,26192,Subscriber,1966,1
2,328,2019-03-01 08:18:30.4030,2019-03-01 08:23:59.0590,3183,Exchange Place,40.716247,-74.033459,3213,Van Vorst Park,40.718489,-74.047727,26226,Subscriber,1971,2
3,95,2019-03-01 11:04:47.2020,2019-03-01 11:06:22.8130,3183,Exchange Place,40.716247,-74.033459,3184,Paulus Hook,40.714145,-74.033552,26204,Subscriber,1985,1
4,169,2019-03-01 11:51:29.4170,2019-03-01 11:54:19.2120,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,29449,Subscriber,1971,1


### Split Gender and Usertype Columns to be Used for Aggregation

In [4]:
# Citi Bike's documentation says that unknown gender is 0, male is 1, and female is 2.
df["gender_unknown"] = df["gender"] == 0
df["gender_unknown"] = df["gender_unknown"].astype(int)

df["gender_male"] = df["gender"] == 1
df["gender_male"] = df["gender_male"].astype(int)

df["gender_female"] = df["gender"] == 2
df["gender_female"] = df["gender_female"].astype(int)

df["subscriber"] = df["usertype"] == "Subscriber"
df["subscriber"] = df["subscriber"].astype(int)

df["customer"] = df["usertype"] == "Customer"
df["customer"] = df["customer"].astype(int)

df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,gender_unknown,gender_male,gender_female,subscriber,customer
0,306,2019-03-01 06:47:08.2110,2019-03-01 06:52:14.7150,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,26272,Subscriber,1989,1,0,1,0,1,0
1,123,2019-03-01 07:30:53.0220,2019-03-01 07:32:56.7770,3183,Exchange Place,40.716247,-74.033459,3184,Paulus Hook,40.714145,-74.033552,26192,Subscriber,1966,1,0,1,0,1,0
2,328,2019-03-01 08:18:30.4030,2019-03-01 08:23:59.0590,3183,Exchange Place,40.716247,-74.033459,3213,Van Vorst Park,40.718489,-74.047727,26226,Subscriber,1971,2,0,0,1,1,0
3,95,2019-03-01 11:04:47.2020,2019-03-01 11:06:22.8130,3183,Exchange Place,40.716247,-74.033459,3184,Paulus Hook,40.714145,-74.033552,26204,Subscriber,1985,1,0,1,0,1,0
4,169,2019-03-01 11:51:29.4170,2019-03-01 11:54:19.2120,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,29449,Subscriber,1971,1,0,1,0,1,0


### Groupby Start Station Info, Aggregate the Rest

In [5]:
# Groupby to get aggregate data for all start stations for the month.
start_station_df = df.groupby(["start station id", "start station name", "start station latitude", "start station longitude"]).sum()
# Only grab necessary columns.
start_station_df = start_station_df[["gender_male", "gender_female", "gender_unknown", "subscriber", "customer"]]
# Reset the index for cleanliness.
start_station_df.reset_index(inplace=True)
# Needed a total uses column.
start_station_df["total_uses"] = start_station_df["subscriber"] + start_station_df["customer"]
# stationtype 1 is for the starting stations.  Used for filtering in Tableau.
start_station_df["stationtype"] = 1
# Rename columns.
start_station_df.columns = ["station_id", "station_name", "station_latitude", "station_longitude", "gender_male", "gender_female", "gender_unknown", "subscriber", "customer", "total_uses", "station_type"]
# Preview result.
start_station_df.head()

,station_id,station_name,station_latitude,station_longitude,gender_male,gender_female,gender_unknown,subscriber,customer,total_uses,station_type
0,3183,Exchange Place,40.716247,-74.033459,831,123,91,967,78,1045,1
1,3184,Paulus Hook,40.714145,-74.033552,384,45,23,422,30,452,1
2,3185,City Hall,40.717733,-74.043845,551,121,35,682,25,707,1
3,3186,Grove St PATH,40.719586,-74.043117,2451,593,69,3066,47,3113,1
4,3187,Warren St,40.721124,-74.038051,393,89,35,492,25,517,1


### Do the Same for End Stations

In [6]:
# Groupby to get aggregate data for all start stations for the month.
end_station_df = df.groupby(["end station id", "end station name", "end station latitude", "end station longitude"]).sum()
# Only grab necessary columns.
end_station_df = end_station_df[["gender_male", "gender_female", "gender_unknown", "subscriber", "customer"]]
# Reset the index for cleanliness.
end_station_df.reset_index(inplace=True)
# Needed a total uses column.
end_station_df["total_uses"] = end_station_df["subscriber"] + end_station_df["customer"]
# stationtype 0 is for the ending stations.  Used for filtering in Tableau.
end_station_df["stationtype"] = 0
# Rename columns.
end_station_df.columns = ["station_id", "station_name", "station_latitude", "station_longitude", "gender_male", "gender_female", "gender_unknown", "subscriber", "customer", "total_uses", "station_type"]
# Preview result.
end_station_df.head()

,station_id,station_name,station_latitude,station_longitude,gender_male,gender_female,gender_unknown,subscriber,customer,total_uses,station_type
0,327,Vesey Pl & River Terrace,40.715338,-74.016584,1,0,0,1,0,1,0
1,459,W 20 St & 11 Ave,40.746745,-74.007756,0,1,0,1,0,1,0
2,513,W 56 St & 10 Ave,40.768254,-73.988639,0,1,0,1,0,1,0
3,514,12 Ave & W 40 St,40.760875,-74.002777,1,0,2,1,2,3,0
4,523,W 38 St & 8 Ave,40.754666,-73.991382,1,0,0,1,0,1,0


### Combine the DataFrames

In [7]:
# Coulda just appended to the previous dataframe, but I wanted a new one for readability.
monthly_df = start_station_df
monthly_df = monthly_df.append(end_station_df)
# Set the date to the first of the month because I don't want to putz with what date formats 
# that Tableau may or may not be able to read.
monthly_df["date"] = f"{month}/01/{year}"
monthly_df.head()

,station_id,station_name,station_latitude,station_longitude,gender_male,gender_female,gender_unknown,subscriber,customer,total_uses,station_type,date
0,3183,Exchange Place,40.716247,-74.033459,831,123,91,967,78,1045,1,03/01/2019
1,3184,Paulus Hook,40.714145,-74.033552,384,45,23,422,30,452,1,03/01/2019
2,3185,City Hall,40.717733,-74.043845,551,121,35,682,25,707,1,03/01/2019
3,3186,Grove St PATH,40.719586,-74.043117,2451,593,69,3066,47,3113,1,03/01/2019
4,3187,Warren St,40.721124,-74.038051,393,89,35,492,25,517,1,03/01/2019


### Write to a New CSV

In [8]:
monthly_df.to_csv(os.path.join("data","aggregated",f"{prefix}{year}{month}-citibike-aggregated.csv"))

### Age

In [9]:
# For the Age CSV I only needed totals for each birth year.
age_df = df.groupby(["birth year"]).sum()
age_df = age_df[["tripduration", "gender_unknown", "gender_male", "gender_female", "subscriber", "customer"]]
age_df = age_df.rename(columns={"tripduration": "tripduration_sum"})
age_df["total_uses"] = age_df["subscriber"] + age_df["customer"]
age_df.reset_index(inplace=True)
age_df.head()

,birth year,tripduration_sum,gender_unknown,gender_male,gender_female,subscriber,customer,total_uses
0,1941,413,0,0,1,1,0,1
1,1947,1414,0,3,0,2,1,3
2,1949,21637,0,47,0,46,1,47
3,1950,9944,0,16,0,16,0,16
4,1951,9815,0,44,0,44,0,44


### Age CSV

In [10]:
age_df.to_csv(os.path.join("data","age",f"{prefix}{year}{month}-citibike-age.csv"))